In [7]:
from Crypto.Cipher import ChaCha20
import struct

def rotate(v, c):
    return ((v << c) & 0xffffffff) | v >> (32 - c)

def quarter_round(x, a, b, c, d):
    x[a] = (x[a] + x[b]) & 0xffffffff
    x[d] = rotate(x[d] ^ x[a], 16)
    x[c] = (x[c] + x[d]) & 0xffffffff
    x[b] = rotate(x[b] ^ x[c], 12)
    x[a] = (x[a] + x[b]) & 0xffffffff
    x[d] = rotate(x[d] ^ x[a], 8)
    x[c] = (x[c] + x[d]) & 0xffffffff
    x[b] = rotate(x[b] ^ x[c], 7)

def yield_chacha20_xor_stream(key, iv, position=0):
    if not isinstance(position, int):
        raise TypeError
    if position & ~0xffffffff:
        raise ValueError('Position is not uint32.')
    if not isinstance(key, bytes):
        raise TypeError
    if not isinstance(iv, bytes):
        raise TypeError
    if len(key) != 32:
        raise ValueError('Key must be 32 bytes long.')
    if len(iv) != 8:
        raise ValueError('Initialization Vector (IV) must be 8 bytes long.')

    ctx = [0] * 16
    ctx[:4] = (1634760805, 857760878, 2036477234, 1797285236)
    ctx[4 : 12] = struct.unpack('<8L', key)
    ctx[12] = ctx[13] = position
    ctx[14 : 16] = struct.unpack('<LL', iv)

    while True:
        x = list(ctx)
        for i in range(10):
            quarter_round(x, 0, 4,  8, 12)
            quarter_round(x, 1, 5,  9, 13)
            quarter_round(x, 2, 6, 10, 14)
            quarter_round(x, 3, 7, 11, 15)
            quarter_round(x, 0, 5, 10, 15)
            quarter_round(x, 1, 6, 11, 12)
            quarter_round(x, 2, 7,  8, 13)
            quarter_round(x, 3, 4,  9, 14)

        for c in struct.pack('<16L', *(
            (x[i] + ctx[i]) & 0xffffffff for i in range(16))):
            yield c

        ctx[12] = (ctx[12] + 1) & 0xffffffff
        if ctx[12] == 0:
            ctx[13] = (ctx[13] + 1) & 0xffffffff

def chacha20_encrypt(data, key, iv=None, position=0):
    if not isinstance(data, bytes):
        raise TypeError('Data must be bytes.')
    if iv is None:
        iv = b'\0' * 8
    if isinstance(key, bytes):
        if not key:
            raise ValueError('Key is empty.')
        if len(key) < 32:
            key = (key * (32 // len(key) + 1))[:32]
        if len(key) > 32:
            raise ValueError('Key is too long.')

    return bytes(a ^ b for a, b in
        zip(data, yield_chacha20_xor_stream(key, iv, position)))

def chacha20_decrypt(data, key, iv=None, position=0):
    decrypted_data = chacha20_encrypt(data, key, iv, position)
    return decrypted_data

def decrypt_image(input_path, output_path, key):
    with open(input_path, 'rb') as encrypted_file:
        data = encrypted_file.read()

    iv = data[:8]
    encrypted_data = data[8:]

    decrypted_data = chacha20_decrypt(encrypted_data, key, iv)

    with open(output_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

# Get input file path from the user
input_path = input("Enter the path to the encrypted image file: ")

# Choose the file name for decryption
decrypted_image_path = 'decrypted_image.png'

# Get key from the user
key = bytes(input("Enter the key (32 bytes): "), 'utf-8')

# Decrypt image
decrypt_image(input_path, decrypted_image_path, key)

print(f"Decrypted image saved to {decrypted_image_path}")

Decrypted image saved to decrypted_image.png
